## Initial Imports

In [40]:
import pandas as pd
import datetime
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns',None)

## Imports

In [15]:
names=['id','created_at','destroyed_at']
servers = pd.read_csv('Data/servers.csv',names=names)
names = ['timestamp','device_id', 'user_id','server_id','connected']
events = pd.read_csv('Data/connectivity_events.csv',names=names)

In [16]:
servers.shape

(188, 3)

In [17]:
events.shape

(9308207, 5)

In [18]:
%%time
combined_df = events.merge(servers,left_on='server_id',right_on='id')
combined_df.shape

CPU times: user 667 ms, sys: 535 ms, total: 1.2 s
Wall time: 1.21 s


(9308207, 8)

In [8]:
combined_df.head()

,timestamp,device_id,user_id,server_id,connected,id,created_at,destroyed_at
0,2015-05-31 14:30:00,100,4,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929
1,2015-05-31 14:30:00,101,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929
2,2015-05-31 14:30:00,102,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929
3,2015-05-31 14:30:00,103,26,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929
4,2015-05-31 14:30:00,104,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929


In [13]:
combined_df['connection_id'] = combined_df.sort_values(['timestamp']).groupby(['device_id']).cumcount()+1
combined_df[combined_df['device_id'] == 100].sort_values(['connection_id'])

,timestamp,device_id,user_id,server_id,connected,id,created_at,destroyed_at,connection_id
0,2015-05-31 14:30:00,100,4,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,1
856,2015-06-02 10:30:00,100,4,2,False,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2
1266,2015-06-03 11:30:00,100,4,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,3
1517,2015-06-04 03:30:00,100,4,2,False,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,4
1657,2015-06-04 13:30:00,100,4,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,5
...,...,...,...,...,...,...,...,...,...
37695,2016-01-20 02:30:00,100,4,2,False,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,334
37740,2016-01-20 09:30:00,100,4,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,335
37766,2016-01-20 12:30:00,100,4,2,False,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,336
37829,2016-01-20 19:30:00,100,4,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,337


In [26]:
def prev_server(df):
    """
    calculate previous server connection's data: previous server id, previous connected status and previous timestamp
    """
    df['prev_server_id']=(df.sort_values(['timestamp'], ascending=True)
                               .groupby(['device_id'])['server_id'].shift(-1))
    df['prev_connected']=(df.sort_values(['timestamp'], ascending=True)
                             .groupby(['device_id'])['connected'].shift(-1)).astype(bool)
    df['prev_timestamp']=(df.sort_values(['timestamp'], ascending=True)
                                             .groupby(['device_id'])['timestamp'].shift(-1))
    df['ignored']=False

def mark_ignored_row(row):
    """
    function to mark rows as ignored, cases:
    1. X offline -> X offline
    2. X offline -> Y offline
    """
    return row['connected']==False and row['prev_connected']==False

    
def process_dataframe(df):
    df['ignored'] = df.apply(lambda row: prepare_final(row), axis=1)

In [8]:
events_merged=events.merge(servers,left_on='server_id',right_on='id')
events_merged.shape[0]

9308207

In [21]:
%%timeit
prev_server(combined_df)

20.6 s ± 515 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
combined_df.head()

,timestamp,device_id,user_id,server_id,connected,id,created_at,destroyed_at,prev_server_id,prev_connected,prev_timestamp,ignored
0,2015-05-31 14:30:00,100,4,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-06-02 10:30:00,False
1,2015-05-31 14:30:00,101,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-05-31 18:30:00,False
2,2015-05-31 14:30:00,102,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-05-31 19:30:00,False
3,2015-05-31 14:30:00,103,26,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-06-01 02:30:00,False
4,2015-05-31 14:30:00,104,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-06-06 02:30:00,False


In [32]:
%%timeit
combined_df['ignored'] = combined_df.apply(mark_ignored_row, axis=1)

2min 57s ± 28.6 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
combined_df[combined_df['ignored']]

,timestamp,device_id,user_id,server_id,connected,id,created_at,destroyed_at,prev_server_id,prev_connected,prev_timestamp,ignored
260729,2017-06-19 22:30:00,17263,2035,2,False,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2017-08-03 22:13:49.725,True
288034,2017-07-19 11:30:00,22711,3072,2,False,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2017-08-03 22:12:18.42,True
289519,2017-07-20 18:30:00,17175,563,2,False,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2017-08-02 23:40:08.182,True
289520,2017-07-20 18:30:00,17185,563,2,False,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2017-08-02 23:37:40.613,True
289521,2017-07-20 18:30:00,17190,563,2,False,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2017-08-02 23:37:44.587,True
...,...,...,...,...,...,...,...,...,...,...,...,...
9094981,2018-02-22 23:58:50.043,52997,540,32,False,32,2018-02-16 22:42:16.20626,2018-02-26 15:52:07.692,32.0,False,2018-02-22 23:59:21.627,True
9095052,2018-02-23 00:01:46.545,6773,119,32,False,32,2018-02-16 22:42:16.20626,2018-02-26 15:52:07.692,32.0,False,2018-02-23 00:22:32.701,True
9095124,2018-02-23 00:04:48.916,20872,2518,32,False,32,2018-02-16 22:42:16.20626,2018-02-26 15:52:07.692,32.0,False,2018-02-23 00:05:56.48,True
9095181,2018-02-23 00:06:57.2,20872,2518,32,False,32,2018-02-16 22:42:16.20626,2018-02-26 15:52:07.692,32.0,False,2018-02-23 00:08:04.008,True


In [47]:
combined_df[combined_df['ignored']].groupby("device_id").count()['ignored'].sort_values(ascending=False)

device_id
9529     723
20872    200
19014    133
19017    127
22703    122
        ... 
29455      1
17934      1
29537      1
29687      1
52997      1
Name: ignored, Length: 7835, dtype: int64

In [64]:
combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'])

In [65]:
combined_df['date'] = combined_df['timestamp'].dt.date

In [66]:
combined_df.head()

,timestamp,device_id,user_id,server_id,connected,id,created_at,destroyed_at,prev_server_id,prev_connected,prev_timestamp,ignored,date
0,2015-05-31 14:30:00,100,4,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-06-02 10:30:00,False,2015-05-31
1,2015-05-31 14:30:00,101,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-05-31 18:30:00,False,2015-05-31
2,2015-05-31 14:30:00,102,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-05-31 19:30:00,False,2015-05-31
3,2015-05-31 14:30:00,103,26,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-06-01 02:30:00,False,2015-05-31
4,2015-05-31 14:30:00,104,1,2,True,2,2015-05-31 14:30:00.000000,2017-08-03 22:42:57.929,2.0,False,2015-06-06 02:30:00,False,2015-05-31


In [78]:
grouped = combined_df[['device_id','date', 'ignored']].groupby(['device_id', 'date']).agg({'ignored':['sum', 'count']})

In [81]:
grouped['sum']

KeyError: 'sum'

combined_df[combined_df['device_id']==19014].sort_values('timestamp')

In [104]:
test['ignored'].value_counts()

False    9621
True      379
Name: ignored, dtype: int64

In [109]:
test[(test['connected']==False) & (test['prev_connected']==False)].shape[0]

4092

In [113]:
%%timeit
process_dataframe(events_merged)

2min 7s ± 3.16 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [134]:
%%timeit
pandarallel.initialize()
events_merged['ignored'] = events_merged.parallel_apply(lambda row: prepare_final(row), axis=1)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process

In [135]:
print(events_merged[(events_merged['connected']==False) & (events_merged['prev_connected']==False)].shape[0])
print(events_merged[events_merged['ignored']==True].shape[0])

379931
379931


In [ ]:
print("end:{}".format(datetime.datetime.now()))